In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
!pip install pulp

In [11]:
warehouses = ["A", "B", "C"] 
 
# Creates a dictionary for the number of units of supply for each supply node 
supply = {"A": 300, "B": 600, "C":600} 
 
# Creates a list of all demand nodes 
projects = ["1", "2", "3"] 
 
# Creates a dictionary for the number of units of demand for each demand node 
demand = {      
    "1": 150, 
    "2": 450, 
    "3": 900, 
} 
 
# Creates a list of costs of each transportation path 
costs = [[5,1,9],
         [4,2,8],
         [8,7,2]] 

from pulp import * 
 
# Creates the 'prob' variable to contain the problem data 
prob = LpProblem("Material Supply Problem", LpMinimize) 
costs = makeDict([warehouses, projects], costs, 0) 

# Creates a list of tuples containing all the possible routes for transport 
Routes = [(w, b) for w in warehouses for b in projects] 
 

# A dictionary called 'Vars' is created to contain the referenced variables(the routes) 
vars = LpVariable.dicts("Route", (warehouses, projects), 0, None, LpInteger) 

#Define Objective Function 
#In this step, we will define the minimum objective function by adding it to the 
#LpProblem object. lpSum(vector)is used here to define multiple linear 
#expressions. It also used list comprehension to add multiple  variables. 
# The minimum objective function is added to 'prob' first 
prob += ( 
    lpSum([vars[w][b] * costs[w][b] for (w, b) in Routes]), 
    "Sum_of_Transporting_Costs", 
) 

# The supply maximum constraints are added to prob for each supply node(warehouses) 
for w in warehouses: 
    prob += ( 
        lpSum([vars[w][b] for b in projects]) <= supply[w], 
        "Sum_of_Products_out_of_warehouses_%s" % w, 
            ) 
 
# The demand minimum constraints are added to prob for each demand node(project) 
for b in projects: 
    prob += ( 
        lpSum([vars[w][b] for w in warehouses]) >= demand[b], 
        "Sum_of_Products_into_projects%s" % b, 
    ) 
# The problem is solved using PuLP's choice of Solver 
prob.solve() 

# Print the variables optimized value 
for v in prob.variables(): 
    print(v.name, "=", v.varValue) 
     
# The optimised objective function value is printed to the screen 
print("Value of Objective Function = ", value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/8ac1a8566af84057bd2d7dd6d07b4648-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/8ac1a8566af84057bd2d7dd6d07b4648-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 57 RHS
At line 64 BOUNDS
At line 74 ENDATA
Problem MODEL has 6 rows, 9 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4800 - 0.00 seconds
Cgl0004I processed model has 6 rows, 9 columns (9 integer (0 of which binary)) and 18 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 4800 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0001I Search completed - best objective 4800, took 0 iterations and 0 nodes (0.01 seconds)
Cbc0035I Maximum dept

/opt/conda/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
